In [1]:
try:
    __IPYTHON__
    USING_IPYTHON = True
    %load_ext autoreload
    %autoreload 2
except NameError:
    USING_IPYTHON = False

#### Argparse

In [2]:
import argparse
ap = argparse.ArgumentParser()
ap.add_argument('project_root', help='')
ap.add_argument('--mrp-data-dir', default='data', help='')
ap.add_argument('--mrp-test-dir', default='src/tests', help='')
ap.add_argument('--tests-fixtures-file', default='fixtures/test.jsonl', help='')

ap.add_argument('--graphviz-sub-dir', default='visualization/graphviz', help='')
ap.add_argument('--train-sub-dir', default='training', help='')
ap.add_argument('--companion-sub-dir', default='companion')
ap.add_argument('--jamr-alignment-file', default='jamr.mrp')

ap.add_argument('--test-input-file', default='evaluation/input.mrp', help='')
ap.add_argument('--test-companion-file', default='evaluation/udpipe.mrp', help='')
ap.add_argument('--allennlp-mrp-json-file-template', default='allennlp-mrp-json-small-{}.jsonl', help='')


ap.add_argument('--mrp-file-extension', default='.mrp')
ap.add_argument('--companion-file-extension', default='.conllu')
ap.add_argument('--graphviz-file-template', default='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.mrp/{}.png')
ap.add_argument('--parse-plot-file-template', default='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.png')

arg_string = """
    /data/proj29_ds1/home/slai/mrp2019
"""
arguments = [arg for arg_line in arg_string.split(r'\\n') for arg in arg_line.split()]

In [3]:
if USING_IPYTHON:
    args = ap.parse_args(arguments)
else:
    args = ap.parse_args()

In [4]:
args

Namespace(allennlp_mrp_json_file_template='allennlp-mrp-json-small-{}.jsonl', companion_file_extension='.conllu', companion_sub_dir='companion', graphviz_file_template='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.mrp/{}.png', graphviz_sub_dir='visualization/graphviz', jamr_alignment_file='jamr.mrp', mrp_data_dir='data', mrp_file_extension='.mrp', mrp_test_dir='src/tests', parse_plot_file_template='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.png', project_root='/data/proj29_ds1/home/slai/mrp2019', test_companion_file='evaluation/udpipe.mrp', test_input_file='evaluation/input.mrp', tests_fixtures_file='fixtures/test.jsonl', train_sub_dir='training')

#### Library imports

In [5]:
import json
import logging
import os
import pprint
import re
import string
from collections import Counter, defaultdict, deque

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import plot_util
import torch
from action_state import mrp_json2parser_states, _generate_parser_action_states
from action_state import ERROR, APPEND, RESOLVE, IGNORE
from preprocessing import (CompanionParseDataset, MrpDataset, JamrAlignmentDataset,
                           read_companion_parse_json_file, read_mrp_json_file, parse2parse_json)            
from torch import nn
from tqdm import tqdm

#### ipython notebook specific imports

In [6]:
if USING_IPYTHON:
    # matplotlib config
    %matplotlib inline

In [7]:
sh = logging.StreamHandler()
formatter = logging.Formatter('%(levelname)-8s [%(name)s:%(lineno)d] %(message)s')
sh.setFormatter(formatter)
logging.basicConfig(level=logging.DEBUG, handlers=[sh])
logger = logging.getLogger(__name__)  # pylint: disable=invalid-name
logger.setLevel(logging.INFO)

### Constants

In [8]:
UNKWOWN = 'UNKWOWN'

### Load data

In [9]:
train_dir = os.path.join(args.project_root, args.mrp_data_dir, args.train_sub_dir)

In [10]:
mrp_dataset = MrpDataset()

In [11]:
frameworks, framework2dataset2mrp_jsons = mrp_dataset.load_mrp_json_dir(
    train_dir, args.mrp_file_extension)

frameworks: 100%|██████████| 5/5 [00:17<00:00,  3.14s/it]t/s]


### Data Preprocessing companion

In [12]:
companion_dir = os.path.join(args.project_root, args.mrp_data_dir, args.companion_sub_dir)

In [13]:
cparse_dataset = CompanionParseDataset()

In [14]:
dataset2cid2parse = cparse_dataset.load_companion_parse_dir(companion_dir, args.companion_file_extension)

INFO     [preprocessing:172] framework amr found
dataset: 100%|██████████| 13/13 [00:01<00:00,  7.50it/s]
INFO     [preprocessing:172] framework dm found
dataset: 100%|██████████| 5/5 [00:05<00:00,  1.31s/it]
INFO     [preprocessing:172] framework ucca found
dataset: 100%|██████████| 6/6 [00:00<00:00, 26.16it/s]


In [15]:
dataset2cid2parse_json = cparse_dataset.convert_parse2parse_json()

In [16]:
dataset2cid2parse.keys()

dict_keys(['amr-guidelines', 'bolt', 'cctv', 'dfa', 'dfb', 'fables', 'lorelei', 'mt09sdl', 'proxy', 'rte', 'wb', 'wiki', 'xinhua', 'wsj', 'ewt'])

In [17]:
# Some data is missing
'20003001' in dataset2cid2parse['wsj']

False

### Load JAMR alignment data

In [18]:
jalignment_dataset = JamrAlignmentDataset()

In [19]:
cid2alignment = jalignment_dataset.load_jamr_alignment_file(os.path.join(
    args.project_root,
    args.mrp_data_dir,
    args.companion_sub_dir,
    args.jamr_alignment_file
))

### Load testing data

In [20]:
test_input_filename = os.path.join(args.project_root, args.mrp_data_dir, args.test_input_file)
test_companion_filename = os.path.join(args.project_root, args.mrp_data_dir, args.test_companion_file)

In [21]:
test_mrp_jsons = read_mrp_json_file(test_input_filename)
test_parse_jsons = read_companion_parse_json_file(test_companion_filename)

In [22]:
parse_json = test_parse_jsons['102990']

In [23]:
mrp_json = framework2dataset2mrp_jsons['psd']['wsj'][1]

In [24]:
# framework = 'ucca'
# dataset = 'wiki'

framework = 'dm'
dataset = 'wsj'

In [25]:
cid = list(dataset2cid2parse_json[dataset].keys())[0]

In [26]:
idx, mrp_json = [
    (idx, mrp_json)
    for idx, mrp_json in enumerate(framework2dataset2mrp_jsons[framework][dataset])
    if mrp_json.get('id') == cid
][0]
idx

15082

In [27]:
parse_json = dataset2cid2parse_json[dataset][cid]

In [28]:
doc = mrp_json['input']

In [29]:
doc

'It is unclear what effect the sale of the shopping centers will have on earnings.'

In [30]:
token_pos = 0
anchors = []
char_pos2tokenized_parse_node_id = []

for node_id, node in enumerate(parse_json.get('nodes')):
    label = node.get('label')
    label_size = len(label)
    while doc[token_pos] == ' ':
        token_pos += 1
        char_pos2tokenized_parse_node_id.append(node_id)
    anchors.append((token_pos, token_pos + label_size))
    char_pos2tokenized_parse_node_id.extend([node_id] * (label_size))
    print(node_id, doc[token_pos: token_pos + label_size], anchors[-1], len(char_pos2tokenized_parse_node_id))
    token_pos += label_size

0 It (0, 2) 2
1 is (3, 5) 5
2 unclear (6, 13) 13
3 what (14, 18) 18
4 effect (19, 25) 25
5 the (26, 29) 29
6 sale (30, 34) 34
7 of (35, 37) 37
8 the (38, 41) 41
9 shopping (42, 50) 50
10 centers (51, 58) 58
11 will (59, 63) 63
12 have (64, 68) 68
13 on (69, 71) 71
14 earnings (72, 80) 80
15 . (80, 81) 81


In [31]:
doc

'It is unclear what effect the sale of the shopping centers will have on earnings.'

In [32]:
len(char_pos2tokenized_parse_node_id)

81

In [33]:
doc = mrp_json['input']

In [34]:
mrp_json['tops']

[2]

In [35]:
mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
    mrp_json, 
    tokenized_parse_nodes=parse_json['nodes'],
)

In [36]:
mrp_meta_data

('It is unclear what effect the sale of the shopping centers will have on earnings.',
 [{'id': 2,
   'label': 'unclear',
   'properties': ['pos', 'frame'],
   'values': ['JJ', 'a:e-h'],
   'anchors': [{'from': 6, 'to': 13}]},
  {'id': 3,
   'label': 'what',
   'properties': ['pos', 'frame'],
   'values': ['WP', 'q:i-h-h'],
   'anchors': [{'from': 14, 'to': 18}]},
  {'id': 4,
   'label': 'effect',
   'properties': ['pos', 'frame'],
   'values': ['NN', 'n:x'],
   'anchors': [{'from': 19, 'to': 25}]},
  {'id': 5,
   'label': 'the',
   'properties': ['pos', 'frame'],
   'values': ['DT', 'q:i-h-h'],
   'anchors': [{'from': 26, 'to': 29}]},
  {'id': 6,
   'label': 'sale',
   'properties': ['pos', 'frame'],
   'values': ['NN', 'n_of:x-i'],
   'anchors': [{'from': 30, 'to': 34}]},
  {'id': 8,
   'label': 'the',
   'properties': ['pos', 'frame'],
   'values': ['DT', 'q:i-h-h'],
   'anchors': [{'from': 38, 'to': 41}]},
  {'id': 9,
   'label': 'shop',
   'properties': ['pos', 'frame'],
   'values

In [37]:
curr_node_ids = mrp_meta_data[-3]
token_states = mrp_meta_data[-2]
actions = mrp_meta_data[-1]

In [38]:
*_, curr_node_ids, token_states, actions = mrp_meta_data

In [39]:
for curr_node_id, action, token_state in zip(curr_node_ids, actions, [[]] + token_states):
    action_type, params = action
    print(curr_node_id, action, token_state)

0 (2, None) []
1 (2, None) []
2 (0, None) []
3 (0, None) [(2, 'unclear')]
3 (1, (1, 0, {'id': 3, 'label': 'what', 'properties': ['pos', 'frame'], 'values': ['WP', 'q:i-h-h'], 'anchors': [{'from': 14, 'to': 18}]}, [[]])) [(2, 'unclear'), (3, 'what')]
4 (0, None) [(2, 'unclear'), (3, [(3, 'what')])]
4 (1, (2, 1, {'id': 4, 'label': 'effect', 'properties': ['pos', 'frame'], 'values': ['NN', 'n:x'], 'anchors': [{'from': 19, 'to': 25}]}, [[{'source': 3, 'target': 4, 'label': 'BV', 'id': 3, 'parent': 4, 'child': 3}], []])) [(2, 'unclear'), (3, [(3, 'what')]), (4, 'effect')]
5 (0, None) [(2, 'unclear'), (4, [(3, [(3, 'what')]), (4, 'effect')])]
5 (1, (1, 0, {'id': 5, 'label': 'the', 'properties': ['pos', 'frame'], 'values': ['DT', 'q:i-h-h'], 'anchors': [{'from': 26, 'to': 29}]}, [[]])) [(2, 'unclear'), (4, [(3, [(3, 'what')]), (4, 'effect')]), (5, 'the')]
6 (0, None) [(2, 'unclear'), (4, [(3, [(3, 'what')]), (4, 'effect')]), (5, [(5, 'the')])]
7 (2, None) [(2, 'unclear'), (4, [(3, [(3, 'what'

In [40]:
companion_parser_states, companion_meta_data = mrp_json2parser_states(
    parse_json,
    mrp_doc=doc,
    tokenized_parse_nodes=parse_json['nodes'],
)

In [41]:
logger.info(args.graphviz_file_template.format(
    framework, dataset, cid))

INFO     [__main__:2] http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/dm/wsj.mrp/20988006.png


In [42]:
mrp_json['input']

'It is unclear what effect the sale of the shopping centers will have on earnings.'

In [43]:
mrp_parser_states

[(2,
  [(2, None), (2, None), (0, None)],
  [],
  [],
  [],
  [(2, 2, None)],
  [(2, 'unclear', 'unclear')]),
 (3,
  [(0, None),
   (1,
    (1,
     0,
     {'id': 3,
      'label': 'what',
      'properties': ['pos', 'frame'],
      'values': ['WP', 'q:i-h-h'],
      'anchors': [{'from': 14, 'to': 18}]},
     [[]]))],
  [3],
  [],
  [4],
  [(2, 2, None), (3, 3, [(3, 3, None)])],
  [(2, 'unclear', 'unclear'), (3, [(3, 'what', 'what')])]),
 (4,
  [(0, None),
   (1,
    (2,
     1,
     {'id': 4,
      'label': 'effect',
      'properties': ['pos', 'frame'],
      'values': ['NN', 'n:x'],
      'anchors': [{'from': 19, 'to': 25}]},
     [[{'source': 3,
        'target': 4,
        'label': 'BV',
        'id': 3,
        'parent': 4,
        'child': 3}],
      []]))],
  [],
  [],
  [],
  [(2, 2, None), (4, 4, [(3, 3, [(3, 3, None)]), (4, 4, None)])],
  [(2, 'unclear', 'unclear'),
   (4, [(3, [(3, 'what', 'what')]), (4, 'effect', 'effect')])]),
 (4,
  [],
  [],
  [],
  [],
  [(2, 2, None)

In [44]:
[(node['id'], node.get('label')) for node in mrp_json['nodes']]

[(2, 'unclear'),
 (3, 'what'),
 (4, 'effect'),
 (5, 'the'),
 (6, 'sale'),
 (8, 'the'),
 (9, 'shop'),
 (10, 'center'),
 (12, 'have'),
 (13, 'on'),
 (14, 'earnings')]

In [45]:
doc

'It is unclear what effect the sale of the shopping centers will have on earnings.'

In [46]:
parse_json['nodes']

[{'id': 0,
  'label': 'It',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['it', 'PRON', 'PRP']},
 {'id': 1,
  'label': 'is',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['be', 'VERB', 'VBZ']},
 {'id': 2,
  'label': 'unclear',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['unclear', 'ADJ', 'JJ']},
 {'id': 3,
  'label': 'what',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['what', 'DET', 'WDT']},
 {'id': 4,
  'label': 'effect',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['effect', 'NOUN', 'NN']},
 {'id': 5,
  'label': 'the',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['the', 'DET', 'DT']},
 {'id': 6,
  'label': 'sale',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['sale', 'NOUN', 'NN']},
 {'id': 7,
  'label': 'of',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['of', 'ADP', 'IN']},
 {'id': 8,
  'label': 'the',
  'properties': ['lemma', 'upos', 'xpos'],
  'values': ['the', 'DET', 'DT']},
 {'id': 9,
  'labe

In [47]:
[(node['id'], node['label']) for node in parse_json['nodes']]

[(0, 'It'),
 (1, 'is'),
 (2, 'unclear'),
 (3, 'what'),
 (4, 'effect'),
 (5, 'the'),
 (6, 'sale'),
 (7, 'of'),
 (8, 'the'),
 (9, 'shopping'),
 (10, 'centers'),
 (11, 'will'),
 (12, 'have'),
 (13, 'on'),
 (14, 'earnings'),
 (15, '.')]

In [48]:
anchors

[(0, 2),
 (3, 5),
 (6, 13),
 (14, 18),
 (19, 25),
 (26, 29),
 (30, 34),
 (35, 37),
 (38, 41),
 (42, 50),
 (51, 58),
 (59, 63),
 (64, 68),
 (69, 71),
 (72, 80),
 (80, 81)]

### Create training instance

In [49]:
total_count = 0
with_parse_count = 0
data_size_limit = 10
ignore_framework_set = {'amr', 'dm', 'psd', 'eds'}
ignore_dataset_set = {'ewt'}

In [50]:
allennlp_tests_fixtures_output_file = os.path.join(
    args.project_root, args.mrp_test_dir, args.tests_fixtures_file)
allennlp_train_output_file = os.path.join(
    args.project_root, args.allennlp_mrp_json_file_template.format('train'))
allennlp_test_output_file = os.path.join(
    args.project_root, args.allennlp_mrp_json_file_template.format('test'))

In [51]:
# Create tests fixture jsonl
fixture_combinations = [
    ('ucca', 'wiki', 70)
]

with open(allennlp_tests_fixtures_output_file, 'w') as wf:
    for framework, dataset, idx in fixture_combinations:
        mrp_json = framework2dataset2mrp_jsons[framework][dataset][idx]
        cid = mrp_json.get('id')
        doc = mrp_json.get('input')
        
        alignment = {}
        if framework == 'amr':
            alignment = cid2alignment[cid]  
        parse_json = dataset2cid2parse_json.get(dataset, {}).get(cid, {})

        if parse_json:
            with_parse_count += 1
            mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
                mrp_json, 
                tokenized_parse_nodes=parse_json['nodes'],
                alignment=alignment,
            )
            companion_parser_states, companion_meta_data = mrp_json2parser_states(
                parse_json, 
                mrp_doc=doc,
                tokenized_parse_nodes=parse_json['nodes'],
            )

            data_instance = {
                'mrp_json': mrp_json,
                'parse_json': parse_json,
                'mrp_parser_states': mrp_parser_states,
                'mrp_meta_data': mrp_meta_data,
                'companion_parser_states': companion_parser_states,
                'companion_meta_data': companion_meta_data,
            }
            json_encoded_instance = json.dumps(data_instance)
            wf.write(json_encoded_instance + '\n')

In [52]:
[state[-1] for state in mrp_parser_states]

[[(0, [(0, 'In', 'In')])],
 [(0, [(0, 'In', 'In')]), (1, [(1, 'the', 'the')])],
 [(0, [(0, 'In', 'In')]),
  (1, [(1, 'the', 'the')]),
  (2, [(2, 'final', 'final')])],
 [(0, [(0, 'In', 'In')]),
  (1, [(1, 'the', 'the')]),
  (2, [(2, 'final', 'final')]),
  (3, [(3, 'minute', 'minute')])],
 [(31,
   [(0, [(0, 'In', 'In')]),
    (1, [(1, 'the', 'the')]),
    (2, [(2, 'final', 'final')]),
    (3, [(3, 'minute', 'minute')])])],
 [(31,
   [(0, [(0, 'In', 'In')]),
    (1, [(1, 'the', 'the')]),
    (2, [(2, 'final', 'final')]),
    (3, [(3, 'minute', 'minute')])]),
  (4, [(4, 'of', 'of')])],
 [(31,
   [(0, [(0, 'In', 'In')]),
    (1, [(1, 'the', 'the')]),
    (2, [(2, 'final', 'final')]),
    (3, [(3, 'minute', 'minute')])]),
  (4, [(4, 'of', 'of')]),
  (5, [(5, 'the', 'the')])],
 [(31,
   [(0, [(0, 'In', 'In')]),
    (1, [(1, 'the', 'the')]),
    (2, [(2, 'final', 'final')]),
    (3, [(3, 'minute', 'minute')])]),
  (4, [(4, 'of', 'of')]),
  (5, [(5, 'the', 'the')]),
  (6, [(6, 'game', 'game')]

In [53]:
mrp_meta_data[-1]

[(0, None),
 (1,
  (1, 0, {'id': 0, 'anchors': [{'from': 0, 'to': 2}], 'label': 'In'}, [[]])),
 (0, None),
 (1,
  (1, 0, {'id': 1, 'anchors': [{'from': 3, 'to': 6}], 'label': 'the'}, [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 2, 'anchors': [{'from': 7, 'to': 12}], 'label': 'final'},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 3, 'anchors': [{'from': 13, 'to': 19}], 'label': 'minute'},
   [[]])),
 (1,
  (4,
   3,
   {'id': 31},
   [[{'source': 31,
      'target': 0,
      'label': 'R',
      'id': 31,
      'parent': 31,
      'child': 0}],
    [{'source': 31,
      'target': 1,
      'label': 'E',
      'id': 28,
      'parent': 31,
      'child': 1}],
    [{'source': 31,
      'target': 2,
      'label': 'E',
      'id': 21,
      'parent': 31,
      'child': 2}],
    [{'source': 31,
      'target': 3,
      'label': 'C',
      'id': 14,
      'parent': 31,
      'child': 3}]])),
 (0, None),
 (1,
  (1, 0, {'id': 4, 'anchors': [{'from': 20, 'to': 22}], 'label': 'of'}, [[]])),

In [54]:
doc

'In the final minute of the game, Johnson had the ball stolen by Celtics center Robert Parish, and then missed two free throws that could have won the game.'

In [55]:
parse_json

{'id': '470004',
 'tops': [9],
 'nodes': [{'id': 0,
   'label': 'In',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['in', 'ADP', 'IN']},
  {'id': 1,
   'label': 'the',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['the', 'DET', 'DT']},
  {'id': 2,
   'label': 'final',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['final', 'ADJ', 'JJ']},
  {'id': 3,
   'label': 'minute',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['minute', 'NOUN', 'NN']},
  {'id': 4,
   'label': 'of',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['of', 'ADP', 'IN']},
  {'id': 5,
   'label': 'the',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['the', 'DET', 'DT']},
  {'id': 6,
   'label': 'game',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': ['game', 'NOUN', 'NN']},
  {'id': 7,
   'label': ',',
   'properties': ['lemma', 'upos', 'xpos'],
   'values': [',', 'PUNCT', ',']},
  {'id': 8,
   'label': 'Johnson',
   'properties': ['lemma', 'up

In [56]:
[n['values'][2] for n in parse_json['nodes']]

['IN',
 'DT',
 'JJ',
 'NN',
 'IN',
 'DT',
 'NN',
 ',',
 'NNP',
 'VBD',
 'DT',
 'NN',
 'VBN',
 'IN',
 'NNPS',
 'NN',
 'NNP',
 'NNP',
 ',',
 'CC',
 'RB',
 'VBD',
 'CD',
 'JJ',
 'NNS',
 'WDT',
 'MD',
 'VB',
 'VBN',
 'DT',
 'NN',
 '.']

In [57]:
# Create train jsonl
if os.path.isfile(allennlp_train_output_file):
    logger.info('allennlp_train_output_file found, stop generation')
else:
    pass
if 1==1:
    data_size = 0
    with open(allennlp_train_output_file, 'w') as wf:
        for _, dataset, mrp_json in tqdm(mrp_dataset.mrp_json_generator(
            ignore_framework_set=ignore_framework_set,
            ignore_dataset_set=ignore_dataset_set,
        )):
            total_count += 1
            if data_size >= data_size_limit:
                break
            cid = mrp_json.get('id')
            doc = mrp_json.get('input')
            
            framework = mrp_json.get('framework')
            alignment = {}
            if framework == 'amr':
                alignment = cid2alignment[cid]  
            parse_json = dataset2cid2parse_json.get(dataset, {}).get(cid, {})
            
            if parse_json:
                mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
                    mrp_json, 
                    tokenized_parse_nodes=parse_json['nodes'],
                    alignment=alignment,
                )
                companion_parser_states, companion_meta_data = mrp_json2parser_states(
                    parse_json, 
                    mrp_doc=doc,
                    tokenized_parse_nodes=parse_json['nodes'],
                )
                
                # Continue if error
                if not mrp_parser_states:
                    continue
                    
                data_size += 1
                logger.info(data_size)
                data_instance = {
                    'mrp_json': mrp_json,
                    'parse_json': parse_json,
                    'mrp_parser_states': mrp_parser_states,
                    'mrp_meta_data': mrp_meta_data,
                    'companion_parser_states': companion_parser_states,
                    'companion_meta_data': companion_meta_data,
                }
                json_encoded_instance = json.dumps(data_instance)
                wf.write(json_encoded_instance + '\n')

                
# Create test jsonl
if os.path.isfile(allennlp_test_output_file):
    logger.info('allennlp_test_output_file found, stop generation')
else:
    pass
if 1==1:
    data_size = 0
    with open(allennlp_test_output_file, 'w') as wf:
        alignment = {}
        for mrp_json in tqdm(test_mrp_jsons):
            
            if data_size >= data_size_limit:
                break
            cid = mrp_json.get('id', '')
            framework = mrp_json.get('framework', '')
            if framework in ignore_framework_set:
                continue
            parse_json = test_parse_jsons[cid]
            doc = parse_json.get('input')
            companion_parser_states, companion_meta_data = mrp_json2parser_states(
                parse_json, 
                mrp_doc=doc,
                tokenized_parse_nodes=parse_json['nodes'],
            )
            if not companion_parser_states:
                continue
            data_size += 1
            logger.info(data_size)
            data_instance = {
                'mrp_json': mrp_json,
                'parse_json': parse_json,
                'companion_parser_states': companion_parser_states,
                'companion_meta_data': companion_meta_data,
            }
            json_encoded_instance = json.dumps(data_instance)
            wf.write(json_encoded_instance + '\n')

INFO     [__main__:3] allennlp_train_output_file found, stop generation
0it [00:00, ?it/s]INFO     [__main__:42] 1
1it [00:00,  8.28it/s]INFO     [__main__:42] 2
2it [00:00,  6.52it/s]INFO     [__main__:42] 3
4it [00:01,  3.54it/s]WARNING  [action_state:132] doc too big
WARNING  [action_state:132] doc too big
INFO     [__main__:42] 4
6it [00:01,  4.59it/s]INFO     [__main__:42] 5
INFO     [__main__:42] 6
8it [00:01,  5.82it/s]INFO     [__main__:42] 7
10it [00:01,  5.31it/s]INFO     [__main__:42] 8
12it [00:02,  4.54it/s]INFO     [__main__:42] 9
14it [00:03,  3.99it/s]WARNING  [action_state:132] doc too big
WARNING  [action_state:132] doc too big
INFO     [__main__:42] 10

INFO     [__main__:57] allennlp_test_output_file found, stop generation
  0%|          | 0/6288 [00:00<?, ?it/s]INFO     [__main__:82] 1
INFO     [__main__:82] 2
INFO     [__main__:82] 3
INFO     [__main__:82] 4
  0%|          | 4/6288 [00:00<02:58, 35.25it/s]INFO     [__main__:82] 5
INFO     [__main__:82] 6
  0%|    

### Test allennlp dataset reader

In [58]:
import torch.optim as optim

from mrp_library.dataset_readers.mrp_jsons import MRPDatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders.embedding import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.modules.feedforward import FeedForward

from allennlp.training.metrics import CategoricalAccuracy

from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

import json
import logging
from typing import Dict

from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import LabelField, TextField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer
from allennlp.models import Model
from overrides import overrides

INFO     [pytorch_pretrained_bert.modeling:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
/home/slai/.pyenv/versions/mrp/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
DEBUG    [allennlp.common.registrable:56] instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
DEBUG    [allennlp.common.registrable:56] instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
DEBUG    [allennlp.common.registrable:56] instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
DEBUG    [allennlp.common.registrable:56] instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>


In [59]:
from mrp_library.dataset_readers.mrp_jsons_actions import MRPDatasetActionReader

In [60]:
reader = MRPDatasetActionReader()

In [61]:
train_dataset = reader.read(cached_path(allennlp_train_output_file))

0it [00:00, ?it/s]INFO     [mrp_library.dataset_readers.mrp_jsons_actions:58] Reading instances from lines in file at: /data/proj29_ds1/home/slai/mrp2019/allennlp-mrp-json-small-train.jsonl
499it [00:00, 7219.34it/s]


In [62]:
test_dataset = reader.read(cached_path(allennlp_train_output_file))
#test_dataset = reader.read(cached_path(allennlp_test_output_file))

0it [00:00, ?it/s]INFO     [mrp_library.dataset_readers.mrp_jsons_actions:58] Reading instances from lines in file at: /data/proj29_ds1/home/slai/mrp2019/allennlp-mrp-json-small-train.jsonl
499it [00:00, 5971.91it/s]


In [63]:
tests_fixtures_dataset = reader.read(cached_path(allennlp_tests_fixtures_output_file))

0it [00:00, ?it/s]INFO     [mrp_library.dataset_readers.mrp_jsons_actions:58] Reading instances from lines in file at: /data/proj29_ds1/home/slai/mrp2019/src/tests/fixtures/test.jsonl
74it [00:00, 3038.16it/s]


In [64]:
vocab = Vocabulary.from_instances(train_dataset + test_dataset + tests_fixtures_dataset)

INFO     [allennlp.data.vocabulary:396] Fitting token dictionary from dataset.
100%|██████████| 1072/1072 [00:00<00:00, 37087.73it/s]


In [65]:
vocab.print_statistics()

INFO     [allennlp.data.vocabulary:664] Printed vocabulary statistics are only for the part of the vocabulary generated from instances. If vocabulary is constructed by extending saved vocabulary with dataset instances, the directly loaded portion won't be considered here.




----Vocabulary Statistics----


Top 10 most frequent tokens in namespace 'word':
	Token: the		Frequency: 2048
	Token: <START-LABEL>		Frequency: 1312
	Token: <END-LABEL>		Frequency: 1312
	Token: to		Frequency: 836
	Token: ,		Frequency: 820
	Token: and		Frequency: 732
	Token: of		Frequency: 528
	Token: a		Frequency: 520
	Token: for		Frequency: 416
	Token: Bowie		Frequency: 360

Top 10 longest tokens in namespace 'word':
	Token: <START-LABEL>		length: 13	Frequency: 1312
	Token: collaboration		length: 13	Frequency: 96
	Token: productivity		length: 12	Frequency: 56
	Token: materialized		length: 12	Frequency: 20
	Token: <END-LABEL>		length: 11	Frequency: 1312
	Token: Switzerland		length: 11	Frequency: 108
	Token: effectively		length: 11	Frequency: 104
	Token: materialize		length: 11	Frequency: 20
	Token: transition		length: 10	Frequency: 124
	Token: compulsion		length: 10	Frequency: 124

Top 10 shortest tokens in namespace 'word':
	Token: “		length: 1	Frequency: 64
	Token: ”		length: 1	Fre

In [66]:
vocab.get_vocab_size('word')

190

In [67]:
vocab.get_vocab_size('pos')

45

In [68]:
vocab.get_vocab_size('label')

2

In [112]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 100

### Test model

In [113]:
from mrp_library.models.generalizer import Generalizer, ActionGeneralizer
from allennlp.nn import InitializerApplicator, RegularizerApplicator, util
from allennlp.nn.activations import Activation
from allennlp.common.params import Params
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.modules.seq2vec_encoders.pytorch_seq2vec_wrapper import PytorchSeq2VecWrapper

In [114]:
word_embedding = Embedding(num_embeddings=vocab.get_vocab_size('word'),
                            embedding_dim=EMBEDDING_DIM)
word_embedder = BasicTextFieldEmbedder({"word": word_embedding})

In [115]:
pos_embedding = Embedding(num_embeddings=vocab.get_vocab_size('pos'),
                            embedding_dim=EMBEDDING_DIM)
pos_embedder = BasicTextFieldEmbedder({"pos": pos_embedding})

In [117]:
encoder = PytorchSeq2VecWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

In [118]:
classifier_params = Params({
  "input_dim": HIDDEN_DIM * 4 * 3,
  "num_layers": 2,
  "hidden_dims": [50, 3],
  "activations": ["sigmoid", "linear"],
  "dropout": [0.2, 0.0]
})

In [119]:
classifier_feedforward = FeedForward.from_params(classifier_params)

INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.modules.feedforward.FeedForward'> from params {'input_dim': 1200, 'num_layers': 2, 'hidden_dims': [50, 3], 'activations': ['sigmoid', 'linear'], 'dropout': [0.2, 0.0]} and extras set()
INFO     [allennlp.common.params:252] input_dim = 1200
INFO     [allennlp.common.params:252] num_layers = 2
INFO     [allennlp.common.params:252] hidden_dims = [50, 3]
INFO     [allennlp.common.params:252] hidden_dims = [50, 3]
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params ['sigmoid', 'linear'] and extras set()
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params sigmoid and extras set()
INFO     [allennlp.common.params:252] type = sigmoid
IN

In [120]:
parse_label = {
    'word': torch.LongTensor(
        [
            [ 1,  0,  3,  7,  2,  9,  4],
            [ 0,  0,  5,  0,  0,  0,  4]
        ]
    )
}
embedded_parse_label = word_embedder(parse_label)

In [121]:
feature_mask = util.get_text_field_mask(parse_label)

In [122]:
encoded_feature = encoder(embedded_parse_label, feature_mask)

In [123]:
encoded_features = [encoded_feature] * 4 * 3

In [124]:
torch.cat(encoded_features, dim=-1).shape

torch.Size([2, 1200])

In [125]:
logits = classifier_feedforward(torch.cat(encoded_features, dim=-1))

In [126]:
logits.shape

torch.Size([2, 3])

In [127]:
label = torch.tensor([1, 0])

In [128]:
loss_func = torch.nn.CrossEntropyLoss()
loss = loss_func(logits, label)

In [129]:
model = ActionGeneralizer(
    vocab=vocab,
    word_embedder=word_embedder,
    pos_embedder=pos_embedder,
    encoder=encoder,
    classifier_feedforward=classifier_feedforward
)

INFO     [allennlp.nn.initializers:293] Initializing parameters
INFO     [allennlp.nn.initializers:309] Done initializing parameters; the following parameters are using their default initialization from their code
INFO     [allennlp.nn.initializers:314]    classifier_feedforward._linear_layers.0.bias
INFO     [allennlp.nn.initializers:314]    classifier_feedforward._linear_layers.0.weight
INFO     [allennlp.nn.initializers:314]    classifier_feedforward._linear_layers.1.bias
INFO     [allennlp.nn.initializers:314]    classifier_feedforward._linear_layers.1.weight
INFO     [allennlp.nn.initializers:314]    encoder._module.bias_hh_l0
INFO     [allennlp.nn.initializers:314]    encoder._module.bias_ih_l0
INFO     [allennlp.nn.initializers:314]    encoder._module.weight_hh_l0
INFO     [allennlp.nn.initializers:314]    encoder._module.weight_ih_l0
INFO     [allennlp.nn.initializers:314]    pos_embedder.token_embedder_pos.weight
INFO     [allennlp.nn.initializers:314]    word_embedder.token_e

In [130]:
optimizer = optim.SGD(model.parameters(), lr=0.01)
cuda_device = -1

In [131]:
iterator = BucketIterator(batch_size=10, sorting_keys=[("curr_parse_node_label", "num_tokens")])
iterator.index_with(vocab)

In [132]:
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=train_dataset,
    patience=10,
    num_epochs=10,
    cuda_device=cuda_device
)

In [133]:
trainer.train()

INFO     [allennlp.training.trainer:465] Beginning training.
INFO     [allennlp.training.trainer:281] Epoch 0/9
INFO     [allennlp.training.trainer:283] Peak CPU memory usage MB: 5572.064
INFO     [allennlp.training.trainer:287] GPU 0 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 1 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 2 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 3 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 4 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 5 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 6 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 7 memory usage MB: 10
INFO     [allennlp.training.trainer:311] Training


  0%|          | 0/50 [00:00<?, ?it/s]DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'nu



accuracy: 0.4051, loss: 1.0657 ||:  16%|█▌        | 8/50 [00:00<00:05,  8.23it/s]DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_upos': {'pos_length': 1, 'num_tokens': 1}, 'curr_parse_node_xpos': {'pos_length': 1, 'num_tokens': 1}, 'prev_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'prev_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}}
DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10
DEBUG    [allennlp.data.iterator

DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10
DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_upos': {'pos_length': 1, 'num_tokens': 1}, 'curr_parse_node_xpos': {'pos_length': 1, 'num_tokens': 1}, 'prev_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'prev_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}}
DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10


accuracy: 0.4972, loss: 1.0006 ||:  36%|███▌ 

DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10


accuracy: 0.5019, loss: 0.9976 ||:  52%|█████▏    | 26/50 [00:02<00:01, 13.95it/s]DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_upos': {'pos_length': 1, 'num_tokens': 1}, 'curr_parse_node_xpos': {'pos_length': 1, 'num_tokens': 1}, 'prev_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'prev_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}}
DEBUG    [allennlp.data.iterato

DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10
DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_upos': {'pos_length': 1, 'num_tokens': 1}, 'curr_parse_node_xpos': {'pos_length': 1, 'num_tokens': 1}, 'prev_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'prev_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}}
DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10


accuracy: 0.5153, loss: 0.9566 ||:  72%|█████

...

DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10
DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_upos': {'pos_length': 1, 'num_tokens': 1}, 'curr_parse_node_xpos': {'pos_length': 1, 'num_tokens': 1}, 'prev_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'prev_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}}
DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10
DEBUG    [allennlp.data.iterators.data_iterator

accuracy: 0.5833, loss: 0.7698 ||:  60%|██████    | 30/50 [00:00<00:00, 55.24it/s]DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_upos': {'pos_length': 1, 'num_tokens': 1}, 'curr_parse_node_xpos': {'pos_length': 1, 'num_tokens': 1}, 'prev_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'prev_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}}
DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10
DEBUG    [allennlp.data.iterators

DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10
DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_upos': {'pos_length': 1, 'num_tokens': 1}, 'curr_parse_node_xpos': {'pos_length': 1, 'num_tokens': 1}, 'prev_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'prev_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}}
DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 10
DEBUG    [allennlp.data.iterators.data_iterator

DEBUG    [allennlp.data.iterators.data_iterator:151] Batch padding lengths: {'curr_parse_node_label': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_lemma': {'word_length': 1, 'num_tokens': 1}, 'curr_parse_node_upos': {'pos_length': 1, 'num_tokens': 1}, 'curr_parse_node_xpos': {'pos_length': 1, 'num_tokens': 1}, 'prev_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'prev_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'prev_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_labels': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_lemmas': {'word_length': 5, 'num_tokens': 5}, 'next_parse_node_uposs': {'pos_length': 5, 'num_tokens': 5}, 'next_parse_node_xposs': {'pos_length': 5, 'num_tokens': 5}}
DEBUG    [allennlp.data.iterators.data_iterator:152] Batch size: 9


accuracy: 0.5772, loss: 0.7680 ||: 100%|██████████| 50/50 [00:00<00:00, 57.63it/s]INFO     [allennlp.training.tens

{'best_epoch': 9,
 'peak_cpu_memory_MB': 5579.912,
 'peak_gpu_0_memory_MB': 10,
 'peak_gpu_1_memory_MB': 11,
 'peak_gpu_2_memory_MB': 11,
 'peak_gpu_3_memory_MB': 10,
 'peak_gpu_4_memory_MB': 11,
 'peak_gpu_5_memory_MB': 10,
 'peak_gpu_6_memory_MB': 11,
 'peak_gpu_7_memory_MB': 10,
 'training_duration': '0:00:46.515180',
 'training_start_epoch': 0,
 'training_epochs': 9,
 'epoch': 9,
 'training_accuracy': 0.5791583166332666,
 'training_loss': 0.7632948362827301,
 'training_cpu_memory_MB': 5579.912,
 'training_gpu_0_memory_MB': 10,
 'training_gpu_1_memory_MB': 11,
 'training_gpu_2_memory_MB': 11,
 'training_gpu_3_memory_MB': 10,
 'training_gpu_4_memory_MB': 11,
 'training_gpu_5_memory_MB': 10,
 'training_gpu_6_memory_MB': 11,
 'training_gpu_7_memory_MB': 10,
 'validation_accuracy': 0.5771543086172345,
 'validation_loss': 0.7680463314056396,
 'best_validation_accuracy': 0.5771543086172345,
 'best_validation_loss': 0.7680463314056396}

In [66]:
class LstmTagger(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()
        
    def forward(self,
                sentence: Dict[str, torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str, torch.Tensor]:
        mask = get_text_field_mask(sentence)
        embeddings = self.word_embeddings(sentence)
        encoder_out = self.encoder(embeddings, mask)
        tag_logits = self.hidden2tag(encoder_out)
        output = {"tag_logits": tag_logits}
        if labels is not None:
            self.accuracy(tag_logits, labels, mask)
            output["loss"] = sequence_cross_entropy_with_logits(tag_logits, labels, mask)
        return output
    
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}

In [67]:
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)

In [71]:
cuda_device = -1
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("sentence", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=train_dataset,
                  test_datase=test_dataset
                  patience=10,
                  num_epochs=1000,
                  cuda_device=cuda_device)

In [72]:
trainer.train()

allennlp.training.trainer - INFO - Beginning training.
allennlp.training.trainer - INFO - Epoch 0/999
allennlp.training.trainer - INFO - Peak CPU memory usage MB: 5777.16
allennlp.training.trainer - INFO - GPU 0 memory usage MB: 0
allennlp.training.trainer - INFO - GPU 1 memory usage MB: 0
allennlp.training.trainer - INFO - GPU 2 memory usage MB: 0
allennlp.training.trainer - INFO - GPU 3 memory usage MB: 0
allennlp.training.trainer - INFO - GPU 4 memory usage MB: 0
allennlp.training.trainer - INFO - GPU 5 memory usage MB: 0
allennlp.training.trainer - INFO - GPU 6 memory usage MB: 0
allennlp.training.trainer - INFO - GPU 7 memory usage MB: 0
allennlp.training.trainer - INFO - Training
  0%|          | 0/25903 [00:00<?, ?it/s]

KeyError: 'sentence'

In [ ]:
dataset2cid2parse_json['wiki']['502000']

In [ ]:
mrp_json

In [ ]:
mrp_parser_states[0]

In [ ]:
with_parse_count, total_count

In [ ]:
class Resolver(nn.Module):
    def __init__(self, vocab_size,vocab_embed_dim):
        super(Resolver, self).__init__()
        self.vocab_embeds = nn.Embedding(vocab_size, vocab_embed_dim)
        self.pos_embeds = nn.Embedding(pos_size, pos_embed_dim)
        self.reset()
    
    def reset(self):
        self.curr_token_id = 0
        self.token_stack = []
        self.token_id2property_dict = {}
        
    def forward(self, token_tensor, lemma_tensor, upos_tensor, xpos_tensor):
        token_id = self.curr_token_id
        token_embeds = self.vocab_embeds(token_tensor)
        
        lemma_embeds = self.vocab_embeds(lemma_tensor)
        upos_embeds = self.pos_embeds(upos_tensor)
        xpos_embeds = self.pos_embeds(xpos_tensor)
        
        features = (token_embed, lemma_embed, upos_embed, xpos_embed)
        
        action_prob = self.choose_action(features)
        action_type = 
        
        if action_type == APPEND:
            self.token_stack.append(token)
            params = None
        elif action_type == RESOLVE:
            
            params = (num_pop, resolve_node, resolved_edges)
        else:
            params = None
        self.curr_token_id += 1
        return action_type, params
        

In [ ]:
torch.LongTensor([12])

In [ ]:
parse_json

In [ ]:
resolver = Resolver()

In [ ]:
mrp_parser_states

In [ ]:
# plot parse json
cid = mrp_json.get('id')
framework = mrp_json.get('framework')

dataset_dir = os.path.join(args.project_root, args.graphviz_sub_dir, dataset)
plot_util.draw_mrp_graphviz(parse_json, dataset_dir)

logger.info(args.graphviz_file_template.format(framework, dataset, cid))
logger.info(args.parse_plot_file_template.format(dataset, cid))

In [ ]:
parser_states, meta_data = mrp_json2parser_states(parse_json, {})
actions = meta_data[-1]

In [ ]:
actions

In [ ]:
for framework in framework2dataset2mrp_jsons:
    logger.info(framework)
    logger.info(list(framework2dataset2mrp_jsons[framework].keys()))

### Test module

In [ ]:
from action_state import mrp_json2parser_states, _generate_parser_action_states

In [ ]:
from action_state import sentence_spliter

In [ ]:
framework, dataset = [
    ('dm', 'wsj'),
    ('psd', 'wsj'),
    ('eds', 'wsj'),
    ('ucca', 'wiki'),
    ('amr', 'wsj'),
    ('amr', 'wiki'),
][0]

mrp_jsons = framework2dataset2mrp_jsons[framework][dataset]
framework, dataset

In [ ]:
mrp_json = mrp_jsons[0]

In [ ]:
# mrp_json = [mrp_json for mrp_json in mrp_jsons if mrp_json.get('id') == '20209013'][0]

In [ ]:
mrp_json['input']

In [ ]:
alignment = {}
if framework == 'amr':
    cid = mrp_json.get('id', '')
    alignment = cid2alignment[cid]

In [ ]:
logger.info(args.graphviz_file_template.format(
    framework, dataset, mrp_json.get('id')))

In [ ]:
parser_states, meta_data = mrp_json2parser_states(mrp_json, alignment)

In [ ]:
import allennlp
from allennlp.data import Instance
from allennlp.data.tokenizers import Tokenizer, WordTokenizer
from allennlp.modules import Seq2SeqEncoder, TimeDistributed, TextFieldEmbedder
from allennlp.data.fields import LabelField, TextField
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer

In [ ]:
wt = WordTokenizer()

In [ ]:
tokenized_doc = wt.tokenize(mrp_json['input'])

In [ ]:
token_indexers = {"tokens": SingleIdTokenIndexer()}

In [ ]:
doc_field = TextField(tokenized_doc, token_indexers)

In [ ]:
instance_fields = {'doc': doc_field}

In [ ]:
instance = Instance(instance_fields)

In [ ]:
doc_field.as_tensor(padding_lengths=2)

In [ ]:
doc_field._indexer_name_to_indexed_token

In [ ]:
tokenized_doc[1].text_id

In [ ]:
allennlp.data.tokenizers.token.Token('Apple').text

In [ ]:
type(tokenized_doc[0])

In [ ]:
mrp_json['input']

In [ ]:
(
    doc,
    nodes,
    node_id2node,
    edge_id2edge,
    top_oriented_edges,
    token_nodes,
    abstract_node_id_set,
    parent_id2indegree,
    parent_id2child_id_set,
    child_id2parent_id_set,
    child_id2edge_id_set,
    parent_id2edge_id_set,
    token_node_id_set,
    actions,
    anchor2token_id,
    parent_child_id2edge_id_set,
) = meta_data

In [ ]:
assert all([edge.get('id') == i for i, edge in enumerate(top_oriented_edges)])

In [ ]:
abstract_node_id_set

In [ ]:
parent_id2edge_id_set

In [ ]:
parent_id2child_id_set

In [ ]:
# node_id2node

In [ ]:
top_oriented_edges[27]

In [ ]:
top_oriented_edges[5]

In [ ]:
tokenized_doc

In [ ]:
nodes

In [ ]:
parser_node_id_set = set()
parser_edge_id_set = set()
for (node_id, actions, edge_state, abstract_node_state, 
     complete_node_state, node_state, token_stack, pending_token_stack) in parser_states:
    parser_node_id_set.add(node_id)
    for edge_id in edge_state:
        parser_edge_id_set.add(edge_id)
    
    node = node_id2node[node_id]
    node_edges = [edge_id2edge[edge_id] for edge_id in edge_state]
    pprint.pprint((
        node.get('id'),
        actions, 
        node.get('label'), 
        [edge.get('label') for edge in node_edges], 
#         abstract_node_state,
        complete_node_state,
        node_state,
        token_stack,
        pending_token_stack,
    ))
    
print({node.get('id', -1) for node in mrp_json.get('nodes')} - parser_node_id_set)
assert len(parser_node_id_set) == len(mrp_json.get('nodes'))
print({edge_id for edge_id, edge in enumerate(mrp_json.get('edges'))} - parser_edge_id_set)
assert len(parser_edge_id_set) == len(mrp_json.get('edges'))

In [ ]:
action_states = [s[1] for s in parser_states]

In [ ]:
token_poss = []
prev_token_pos = 0
for token in sentence_spliter(doc):
    token_poss.append((prev_token_pos, prev_token_pos + len(token)))
    prev_token_pos += len(token) + 1

In [ ]:
list(sentence_spliter(doc))

In [ ]:
token_poss

In [ ]:
[n['anchors'] for n in nodes]

In [ ]:
nodes

In [ ]:
nodes = mrp_json['nodes']

In [ ]:
logger.setLevel(logging.INFO)

In [ ]:
num_pops = []
error_num = 0
for i, mrp_json in tqdm(enumerate(mrp_jsons)):
#     print(i)
    parser_states, meta_data = mrp_json2parser_states(mrp_json, framework, alignment)
    if not parser_states:
        logger.info(i)
        error_num += 1
        continue
    action_states = [s[1] for s in parser_states]
    for action_state in action_states:
        for action in action_state:
            action_type, arg = action
            if action_type == RESOLVE:
                num_pop = arg
                num_pops.append(num_pop)